In [1]:
from tensor import Tensor 
import numpy as np

In [ ]:
a = Tensor(np.arange(4).reshape((2,2)))
b = Tensor(np.arange(4).reshape((2,2)))

c = a @ b
print(a,b,c)


tensor: [[0 1]
 [2 3]], grad: [[0 0]
 [0 0]], op:None tensor: [[0 1]
 [2 3]], grad: [[0 0]
 [0 0]], op:None tensor: [[0 2]
 [4 6]], grad: [[0 0]
 [0 0]], op:+
